In [1]:
pip install chess

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pygame

Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
import pygame
import chess
import math
import random
import chess.polyglot
import torch
import time
import copy

pygame 2.5.0 (SDL 2.28.0, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
#initialise display
X = 800
Y = 800
scrn = pygame.display.set_mode((X, Y))
pygame.init()
reader = chess.polyglot.open_reader('baron30.bin')
#basic colours
WHITE = (255, 255, 255)
GREY = (128, 128, 128)
YELLOW = (204, 204, 0)
BLUE = (50, 255, 255)
BLACK = (0, 0, 0)


piece_width = 100
piece_height = 100

# Load and resize the piece images
pieces = {
    'p': pygame.transform.scale(pygame.image.load('images/b_p.png').convert(), (piece_width, piece_height)),
    'n': pygame.transform.scale(pygame.image.load('images/b_n.png').convert(), (piece_width, piece_height)),
    'b': pygame.transform.scale(pygame.image.load('images/b_b.png').convert(), (piece_width, piece_height)),
    'r': pygame.transform.scale(pygame.image.load('images/b_r.png').convert(), (piece_width, piece_height)),
    'q': pygame.transform.scale(pygame.image.load('images/b_q.png').convert(), (piece_width, piece_height)),
    'k': pygame.transform.scale(pygame.image.load('images/b_k.png').convert(), (piece_width, piece_height)),
    'P': pygame.transform.scale(pygame.image.load('images/w_p.png').convert(), (piece_width, piece_height)),
    'N': pygame.transform.scale(pygame.image.load('images/w_n.png').convert(), (piece_width, piece_height)),
    'B': pygame.transform.scale(pygame.image.load('images/w_b.png').convert(), (piece_width, piece_height)),
    'R': pygame.transform.scale(pygame.image.load('images/w_r.png').convert(), (piece_width, piece_height)),
    'Q': pygame.transform.scale(pygame.image.load('images/w_q.png').convert(), (piece_width, piece_height)),
    'K': pygame.transform.scale(pygame.image.load('images/w_k.png').convert(), (piece_width, piece_height)),
}

def update(scrn,board):
    '''
    updates the screen basis the board class
    '''
    
    for i in range(64):
        piece = board.piece_at(i)
        if piece == None:
            pass
        else:
            scrn.blit(pieces[str(piece)],((i%8)*100,700-(i//8)*100))
    
    for i in range(7):
        i=i+1
        pygame.draw.line(scrn,WHITE,(0,i*100),(800,i*100))
        pygame.draw.line(scrn,WHITE,(i*100,0),(i*100,800))

    pygame.display.flip()


In [5]:
def main(BOARD):
    scrn.fill(BLACK)
    pygame.display.set_caption('Chess')
    index_moves = []
    status = True
    while (status):
        update(scrn,BOARD)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                status = False

            if event.type == pygame.MOUSEBUTTONDOWN:
                scrn.fill(BLACK)
                pos = pygame.mouse.get_pos()
                square = (math.floor(pos[0]/100),math.floor(pos[1]/100))
                index = (7-square[1])*8+(square[0])
                if index in index_moves: 
                    
                    move = moves[index_moves.index(index)]
                    
                    BOARD.push(move)
                    index=None
                    index_moves = []
                else:
                    #check the square that is clicked
                    piece = BOARD.piece_at(index)
                    #if empty pass
                    if piece == None:
                        
                        pass
                    else:
                        all_moves = list(BOARD.legal_moves)
                        moves = []
                        for m in all_moves:
                            if m.from_square == index:                
                                moves.append(m)
                                t = m.to_square
                                TX1 = 100*(t%8)
                                TY1 = 100*(7-t//8)
                                pygame.draw.rect(scrn,BLUE,pygame.Rect(TX1,TY1,100,100),5)
                        
                        index_moves = [a.to_square for a in moves]
        if BOARD.outcome() != None:
            print(BOARD.outcome())
            status = False
            print(BOARD)
    pygame.quit()

def main_one_agent(BOARD,agent,agent_color):
    scrn.fill(BLACK)
    #name window
    pygame.display.set_caption('Chess')
    index_moves = []
    subtree = [[]]
    status = True
    while (status):
        update(scrn,BOARD)  
        if BOARD.turn==agent_color:
            start_time = time.time()
            tree = [[[BOARD, [0], eval_board(BOARD), 0, 0, 0]]]
            move, layers = agent(tree,subtree)
            BOARD.push(move)
            scrn.fill(BLACK)
            print("time used:", time.time()-start_time, "seconds")
        else:

            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    status = False
                if event.type == pygame.MOUSEBUTTONDOWN:
                    scrn.fill(BLACK)
                    pos = pygame.mouse.get_pos()
                    square = (math.floor(pos[0]/100),math.floor(pos[1]/100))
                    index = (7-square[1])*8+(square[0])
                    if index in index_moves:     
                        move = moves[index_moves.index(index)]
                        #print(BOARD)
                        #print(move)
                        BOARD.push(move)
                        index=None
                        index_moves = []
                    else:                   
                        piece = BOARD.piece_at(index)                    
                        if piece == None:
                            pass
                        else:

                            all_moves = list(BOARD.legal_moves)
                            moves = []
                            for m in all_moves:
                                if m.from_square == index:          
                                    moves.append(m)
                                    t = m.to_square
                                    TX1 = 100*(t%8)
                                    TY1 = 100*(7-t//8)
                                    pygame.draw.rect(scrn,BLUE,pygame.Rect(TX1,TY1,100,100),5)
                            #print(moves)
                            index_moves = [a.to_square for a in moves]
        if BOARD.outcome() != None:
            print(BOARD.outcome())
            status = False
            print(BOARD)
    pygame.quit()

In [6]:
#game state analysis
def pseudoStep(a,b,c,e,x):
    return(a+(b-a)*(np.arctan((x-c)*e)/np.pi + 1/2)) # The higher is e, the greater is the slop
def smooth_indicator(x, a, b):
    e = 10/(b-a)
    left = pseudoStep(0,1,a, e,x)
    right = pseudoStep(1,0,b, e,x)
    return (left+right-1)*100
def total_value(board):
    values = [1, 2.6, 2.7, 4.1, 7.8, 15.4]
    total = 0
    piece_map = board.piece_map()
    for square, piece in piece_map.items():
        piece_type = piece.piece_type
        piece_value = values[piece_type-1]
        total += piece_value
    return(total)
def game_stage(value):
    #100 =>0
    #mid game tipping point : 90
    #end game tipping point : 60
    #king_pawn tipping point : 42
    # prediction : probability of being at opening, mid, end, king-pawn
    prediction = np.array([smooth_indicator(value,90, 100),smooth_indicator(value,60, 90),smooth_indicator(value,42, 60),smooth_indicator(value,0, 42)])
    norm = np.sum(prediction)
    prediction = 100*prediction/norm
    prediction = [int(p) / 100 for p in prediction]
    return(prediction)

In [7]:
#position analysis
def is_passing_pawn(board, square, color):
    rank = chess.square_rank(square)
    file = chess.square_file(square)
    if file == 0:
         n_files = [0,1]
    if file == 7:
        n_files = [6,7]
    else:
        n_files = [file-1, file, file+1]
    if (color == True):
        for f in n_files:
            for r in range(rank+1, 8):
                square = rank * 8 + file
                if board.piece_at(square) == chess.Piece(chess.PAWN, not color):
                    return(False)
        return(True)
    else:
        for f in n_files:
            for r in range(0, rank):
                square = rank * 8 + file
                if board.piece_at(square) == chess.Piece(chess.PAWN, not color):
                    return(False)
        return(True)

    return False
def pawn_value(n, prediction, passing):
    value = np.array([[1,1,1,1.05,0.95,0.7,1.5,6], [1,1,1,1.05,1,0.8,1.8,6], [1,1,1.1,1.2,1.3,1.4,2.1,6], [1,1,1.2,1.4,1.6,1.9,2.8,6]])
    passing_value = np.array([[1,1,1,1,0.95,1.2,1.5,6], [1,1,1,1.05,1,0.8,1.8,6], [1,1,1.2,1.4,1.6,1.8,2.1,6], [1,1,1.3,1.6,1.9,2.3,2.8,6]])
    if passing:
        return np.sum(passing_value[:,n]*prediction)
    else:
        return np.sum(value[:,n]*prediction)
def king_value(file, n, prediction):
    file_value = np.array([[0.2,0.4,0.3,0,0,0.3,0.4,0.2],[0.2,0.8,0.8,0,0,0.8,0.8,0.2],[0,0.3,0.5,0.6,0.6,0.5,0.3,0],[0,0.3,0.5,0.5,0.5,0.5,0.3,0]])
    rank_value = np.array([[2, 1.7, 1.4, 1, 0.6, 0, 0,0],[2, 1.8, 1.5, 1.1, 0.6, 0, 0,0],[1.1, 1.3, 1.5, 1.7, 1.8, 1.6, 1.4, 1.2],[0.7, 1, 1.4, 1.7, 1.9, 2, 1.2,0.7]])
    return np.sum(file_value[:,file]*prediction + rank_value[:,n]*prediction)


In [8]:
#eval
def eval_board(board):
    #will be replaced by a deep network
    #tensor = board_to_tensor(board)[0]
    score = 0
    if board.is_checkmate() and board.turn:
        return(-1000)
    if board.is_checkmate() and board.turn == False:
        return(1000)
    if board.is_stalemate():
        return(0)
    if board.is_insufficient_material():
        return(0)
    if board.has_insufficient_material(board.turn):
        return(-100)
    if board.has_insufficient_material(board.turn == False):
        return(100)
    values = [1, 2.6, 2.7, 4.1, 7.8, 15.4]
    piece_map = board.piece_map()
    prediction = game_stage(total_value(board))
    for square, piece in piece_map.items():
        piece_type = piece.piece_type
        color = (1 if piece.color else -1)
        piece_value = values[piece_type-1] * color
        if piece_type == chess.PAWN:
            n = chess.square_rank(square)
            if piece.color == False:
                n = 7-n
            piece_value = pawn_value(n, prediction, is_passing_pawn(board, square, piece.color))* color
        if piece_type == chess.KING:
            n = chess.square_rank(square)
            if piece.color == False:
                n = 7-n
            piece_value += king_value(chess.square_file(square), n, prediction) * color
        score += piece_value
        valid_squares = list(board.attacks(square))
        for s in valid_squares:
            score += 0.05*color
    turn = (1 if board.turn else -1)
    score += (0.25 if board.has_kingside_castling_rights(True) else 0)
    score += (-0.25 if board.has_kingside_castling_rights(False) else 0)
    score += (0.25 if board.has_queenside_castling_rights(True) else 0)
    score += (-0.25 if board.has_queenside_castling_rights(False) else 0)
    return int(score*100)/100

def stable_eval(b, n):
    value = eval_board(b)
    best = value
    m = 0
    B = b.copy()
    if n<=0:
        return(value)
    moves = list(b.legal_moves)
    for move in moves:
        if b.is_capture(move):
            copy = b.copy()
            copy.push(move)
            v = stable_eval(copy, n-1)
            gain = (v-value)*(1 if b.turn else -1)
            if (gain>1) and (gain > m) :
                m = gain
                best = v
    return(best)



In [9]:
#deep eval
def board_to_tensor(board):
    tensor = np.zeros((1,15, 8, 8), dtype=np.float32)
    values = [1, 2.6, 2.7, 4.1, 7.8, 15.4]
    piece_map = board.piece_map()
    prediction = game_stage(total_value(board))
    for square, piece in piece_map.items():
        piece_type = piece.piece_type
        color = (1 if piece.color else -1)
        piece_value = values[piece_type-1] * color
        layer = int(piece_type-1 + 3*(1+color))
        if piece_type == chess.PAWN:
            n = chess.square_rank(square)
            if piece.color == False:
                n = 7-n
            piece_value = pawn_value(n, prediction, is_passing_pawn(board, square, piece.color))
        if piece_type == chess.KING:
            n = chess.square_rank(square)
            if piece.color == False:
                n = 7-n
            piece_value += king_value(chess.square_file(square), n, prediction)
        tensor[0,layer, chess.square_rank(square), chess.square_file(square)] = piece_value
        valid_squares = list(board.attacks(square))
        for s in valid_squares:
            tensor[0,layer, chess.square_rank(s), chess.square_file(s)] += 0.05*color
    turn = (1 if board.turn else -1)
    tensor[0, 12, 0, 0] = (0.25 if board.has_kingside_castling_rights(True) else 0)
    tensor[0, 12, -1, 0] = (-0.25 if board.has_kingside_castling_rights(False) else 0)
    tensor[0, 12, 0, -1] = (0.25 if board.has_queenside_castling_rights(True) else 0)
    tensor[0, 12, -1, -1] = (-0.25 if board.has_queenside_castling_rights(False) else 0)
    tensor[0, 13, :, :] = turn/640
    if board.is_checkmate():
        tensor[0, 14, :, :] = -turn
    return torch.from_numpy(tensor)

def fen_to_tensor(fen):
    board = chess.Board(fen)
    return(board_to_tensor(board))

In [10]:
#tree[layer,...]
#layer[nodes,...]
#initial tree = [[[chess.Board(), [0], value, 0 ,width[1], width[2]/a, 0]]]
#node [board, origin, value, up_propagation_value, fertility, children fertility, recycle]
def darwinian_tree(tree, subtree, parameters, eval_board, color):
    depth = parameters.depth
    lib = []
    moves = list(b.legal_moves)
    if len(moves) ==0:
        return(chess.Move.null(), [[]])
    else :
        all_moves = list(reader.find_all(b))
        
        if all_moves == []:
            pass
        else:
            random.shuffle(all_moves)
            opening_move = all_moves[0]
            return (opening_move.move, [[]])
        #sub, id = is_board_in_list(b, tree[-1])
        if False:
            print("recycle")
            layers = subtree(layers, id)
            depth = 4
            width = parameters.width[:-4]
        else:
            tree[0][0][4] = parameters.width[1]
            tree[0][0][5] = parameters.width[2]
            print("-------------")
            print("initial")
            print("initial board value = " + str(eval_board(b, 7)))
            print("game_stage = " + str(game_stage(total_value(b))))
            print(b)
            print("exploration stage")
            for i in range(depth):       
                side = ((i%2 == 0)== color)  #if color == False, i ==0 : side = False
                print("reproducing")
                children = reproduce(tree[-1], eval_board, i, depth, side)
                print("selecting children")
                survived = select(children, side, eval_board, i, parameters)
            
                if i < depth-2:
                    print("hunting")
                    """if side:
                        a = len(tree[1])
                        survived = group_with_relatives(survived, 1)
                        hunters = []
                        for k in range(len(survived)):
                            hunters += hunt(survived[k], i+1, parameters, side)
                    else: """ 
                    hunters = hunt(survived, i+1, parameters, side)
                    
                    tree.append(hunters.copy())
                else:
                    tree.append(survived.copy())
                if i==0:
                    a = len(tree[-1])
                    for k in range(a):
                        tree[-1][k][4] = int(tree[-1][k][4]/a)
                        
                print("exploration " + str((i+1)*100/depth) + " %")
            length = []
            for t in tree:
                length.append(len(t))
            print(length)
            print("up propagating")
            for k in range(len(tree[-1])):
                tree[-1][k][3] = tree[-1][k][2]
            best_move, l = up_propagate(tree.copy(), color==(depth%2==1), [])
            print("showing steps")
            print_steps(l)
            length = []
            for t in tree:
                length.append(len(t))
            print(length)
            return(best_move, tree[2:])
            
            
            

In [11]:
#reproduction
def reproduce(parents, eval_board, I ,depth, side):
    children = []
    for k in range(len(parents)):
        #if k%10==0:
            #print("reproduction : " + str(k/len(parents)*100) + " %")
        b = parents[k][0]
        parent_value = parents[k][2]
        moves = list(b.legal_moves)
        temp = []
        if len(moves)==0:
            children.append([[b.copy(), parents[k][1]+ [0], eval_board(b, 0), 0, parents[k][5], parents[k][4] ,0]])
        else:
            for i in range(len(moves)):
                copy = b.copy()
                copy.push(moves[i])
                v = eval_board(copy, 0)
                temp.append([copy, parents[k][1]+ [i], v, 0, parents[k][5],parents[k][4] ,0])
            children.append(temp.copy())
    return(children)

In [12]:
#children selection
def select(children, side, eval_board, I, parameters):
    survived = []
    killed = 0
    for k in range(len(children)):
        s, k = keep(children[k], children[k][0][5], eval_board, I, parameters, side)
        if I == parameters.depth-1:
            for i in range(len(s)):
                if s[i][6]==0:
                    s[i][2] = eval_board(s[i][0], parameters.depth-4)          
        survived += s
        killed +=k
    print("survived: " + str(len(survived)))
    print("killed: " + str(killed))
    return(survived)
def keep(L, n, eval_board, I, parameters, side):
    depth = len(L[0][1])-2
    if len(L)<= n:
        return L, 0
    else:
        L.sort(key=lambda l: l[2], reverse=side)
        w2 = int(np.ceil((1-parameters.risk[depth])*n))
        if I ==parameters.depth:
            for k in range(w2):
                b = L[k][0]
                move = b.peek()
                if (b.is_attacked_by(b.turn, move.from_square) or b.is_capture(move)):
                    L[k][2] = eval_board(b, parameters.depth)
                    L[k][6] = 1  
        p = L[:w2]
        rest = L[w2:]
        l = len(rest)
        if l <=1:
            return L[:n], len(L)-n
        rate = 1/parameters.contrast[depth]**(1/(l-1))
        weights = [rate ** i for i in range(l)]
        s = unique(random.choices(rest, weights, k=n - w2))
        survived = p + s
        return survived, len(L)-len(survived)
def unique(L):
    unique_dict = {}
    for sublist in L:
        value = sublist[0].fen()
        if value not in unique_dict:
            unique_dict[value] = sublist
    unique_list = list(unique_dict.values())
    return unique_list

In [13]:
#hunt
def sort_hunters(groups, depth, side):
    if depth<0:
        return(groups)
    else:
        for k in range(len(groups)):
            groups[k] = [0, groups[k][0][1][:-1], groups[k]]
        groups = group_with_relatives(groups, depth)
        #groups = [family,family,...]
        #family = [[0, position, sorted_list], [0, position, sorted_list],[0, position, sorted_list]...]
        #sorted_list = [[board, position, score...], [board, position, score...], [board, position, score...]]
        for k in range(len(groups)): #pour chaque famille
            family = groups[k]
            new = []
            ind = 0 #On commence par prendre le meilleur
            p = True
            while p:
                l = []
                for i in range(len(family)): 
                    if ind < len(family[i][2]):
                        l.append(family[i][2][ind])
                if l!=[]:
                    l.sort(key=lambda l: l[2], reverse=side)
                    new +=l
                    ind += 1
                else:   
                    p = False   
            groups[k] = new
        return(sort_hunters(groups, depth-1, side==False))
def score_hunters(hunters, width, contrast):
    l = len(hunters)
    if l==0:
        return([])
    elif l ==1:
        hunters[0][4] = width
        return(hunters)
    elif l==2:
        rate = 1/contrast
    else:    
        rate = 1/contrast**(1/(l-1))
    weights = [rate ** (i**0.4*l**0.6) for i in range(l)]
    norm = width/sum(weights)
    for k in range(l):
        hunters[k][5] = norm*weights[k]
    return(hunters)
def hunt(hunters, tree_depth, parameters, side): 
    width = parameters.width[tree_depth+2]
    print(width)
    contrast = parameters.contrast[tree_depth]
    print(contrast)
    for k in range(len(hunters)):
        hunters[k] = [hunters[k]]
    sorted_hunters = sort_hunters(hunters, tree_depth-1, side)[0]
    scored_hunters = score_hunters(sorted_hunters, width, contrast)
    #flatten = flatten_list(group_hunters(hunters, depths, -int(width[tree_depth+1]/a), parameters.contrast, side))
    
    approxed_hunters = approx(scored_hunters)
    a = [e[5] for e in approxed_hunters if e[5]>0.3]
    print(a)
    print(sum(a))
    return(approxed_hunters)
def approx(hunters):
    keep = []
    for k in range(len(hunters)):
        f = hunters[k][5]
        if f <3:
            f = 3*np.random.binomial(1,int(f*100)/300)
        else:
            i = np.floor(f)
            p = int((f-i)*100)/300
            f = i + 3*np.random.binomial(1,p)
        if f>=2:
            hunters[k][5] = int(f)
            keep.append(k)
    hunters = [hunters[i] for i in keep] 
    return(hunters)

def group_with_relatives(L, depth):
    result = {}
    for l in L:
        prefix = l[1][:depth+1]
        if str(prefix) in result:
            result[str(prefix)].append(l)
        else:
            result[str(prefix)] = [l]
    return (list(result.values()))
def flatten_list(lst):
    flattened_list = []
    for item in lst:
        if isinstance(item[0], list):
            flattened_list.extend(flatten_list(item))
        else:
            flattened_list.append(item)
    return flattened_list

In [14]:
#up propagation
def keep_best(children, side):
    depth = len(children[0][1])
    groups = group_with_relatives(children, depth-2)
    best = []
    for group in groups:
        if side:
            best.append(max(group, key=lambda x: x[3]))
        else:
            best.append(min(group, key=lambda x: x[3]))
    return(best)
"""def clean_tree(tree):
    depth = len(tree)
    for k in range(depth-2):
        parent = 
        children = {str(l2[1][:-1]): l2[2] for l2 in tree[depth-k-1]}
        for parent in tree[depth-k-2]:
            if str()"""
    
def up_propagate(tree, side, l):
    if len(tree)<=2:
        L2 = keep_best(tree[-1], side)
        l.append(L2)
        b = L2[0][0].copy()
        return b.pop(), l
    else:
        L2 = keep_best(tree[-1], side)          
        l.append(L2)
        dict_l2 = {str(l2[1][:-1]): l2[3] for l2 in L2}
        new = tree[-2].copy()
        for l1 in new:
            if str(l1[1]) in dict_l2:
                l1[3] = dict_l2[str(l1[1])]
            else : 
                l1[3] = 9999
        tree[-2] = [n for n in new if n[3] != 9999]
        return(up_propagate(tree[:-1], side==False, l))

def print_steps(l):
    l.reverse()
    next_ = [0]
    for k in range(len(l)):
        P = True
        for e in l[k]:
            if e[1][:-1]==next_:
                next_ = e[1]
                print(e[0])
                print("value = " + str(e[2]))
                P = False
                continue
        if P:
            print("not found " + str(next_))

In [15]:
#parameters
class Parameters:
    def __init__(self, depth, width, risk, contrast, agressivity):
        self.depth = depth
        self.width = width
        self.risk = risk
        self.contrast = contrast
        self.agressivity = agressivity
    def random_modify(self):
        self.width = [w * random.uniform(0.9, 1.1) for w in self.width]
        self.risk = [r * random.uniform(0.9, 1.1) for r in self.risk]
        self.contrast = [c * random.uniform(0.9, 1.1) for c in self.contrast]
        self.aggressivity *= random.uniform(0.9, 1.1)

In [16]:
def play_min_maxN(tree, subtree):
    depth = 6
    width = [1, 30, 200, 400, 700, 600, 350]
    risk = [0.4,0.4,0.3,0.3,0.25,0.25, 0.2] # risk[0] selection pour les enfants de la racine
    contrast = [8,15,25,60,50,30,20] #contrast[1] contraste pour la sélection et distribution de ressource pour la couche 1 
    agressivity = 0.5
    parameters = Parameters(depth, width, risk, contrast, agressivity)
    color = False
    return darwinian_tree(tree, subtree, parameters, stable_eval, color)

In [17]:
b = chess.Board()
main_one_agent(b,play_min_maxN,False)

time used: 0.0015151500701904297 seconds
time used: 0.0009989738464355469 seconds
-------------
initial
initial board value = -0.09
game_stage = [0.84, 0.12, 0.0, 0.01]
r n b q k b . r
p p p p . p p p
. . . . p n . .
. . . . . . . .
. . . . P . . .
. . . . . . . .
P P P P Q P P P
R N B . K B N R
exploration stage
reproducing
selecting children
survived: 28
killed: 0
hunting
400
15
[76, 39, 28, 24, 21, 18, 19, 15, 13, 12, 14, 11, 10, 9, 9, 8, 8, 7, 7, 6, 6, 6, 8, 5, 5, 5, 5, 4]
398
exploration 16.666666666666668 %
reproducing
selecting children
survived: 193
killed: 613
hunting
700
25
[27, 18, 16, 14, 13, 15, 12, 11, 11, 10, 10, 9, 9, 9, 8, 11, 8, 8, 7, 10, 7, 7, 7, 6, 9, 6, 6, 6, 6, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 7, 7, 4, 4, 4, 4, 4, 4, 7, 4, 4, 4, 7, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 6, 6, 3, 3, 3, 3, 6, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,

In [18]:
"""def group_hunters(hunters, depths, width, contrast, side):
    if depths == []:
        return(hunters)
    else:
        depth = depths.pop()
        groups = group_with_relatives(hunters, depth)
        for k in range(len(groups)):
            group = groups[k]
            if width <0:
                width = -width
                #print(width)
            else:
                width = 0
                for g in group:
                    width +=g[4]
            if len(group) <=2:
                for i in range(len(group)):
                    group[i][4] = width/len(group)
                ranked_hunters = group
            else:
                ranked_hunters = rank_hunters(group, depths, width, contrast*np.log(len(group))**4*0.2, side)
            #print(ranked_hunters)
            groups[k] = group_hunters(ranked_hunters, depths, width, contrast, side)
    return(groups)
def rank_hunters(hunters, depths, width, contrast, side):
    hunters.sort(key=lambda l: l[2], reverse=side)
    l = len(hunters)
    S = 0
    for k in range(l):
        s = hunters[k][4]*contrast**((l-k)/l)
        hunters[k][4] = s
        S += s
    norm = width/S
    for k in range(l):
        hunters[k][4] *= norm
    return(hunters)"""

'def group_hunters(hunters, depths, width, contrast, side):\n    if depths == []:\n        return(hunters)\n    else:\n        depth = depths.pop()\n        groups = group_with_relatives(hunters, depth)\n        for k in range(len(groups)):\n            group = groups[k]\n            if width <0:\n                width = -width\n                #print(width)\n            else:\n                width = 0\n                for g in group:\n                    width +=g[4]\n            if len(group) <=2:\n                for i in range(len(group)):\n                    group[i][4] = width/len(group)\n                ranked_hunters = group\n            else:\n                ranked_hunters = rank_hunters(group, depths, width, contrast*np.log(len(group))**4*0.2, side)\n            #print(ranked_hunters)\n            groups[k] = group_hunters(ranked_hunters, depths, width, contrast, side)\n    return(groups)\ndef rank_hunters(hunters, depths, width, contrast, side):\n    hunters.sort(key=lamb

In [19]:
prediction = np.array([0,1,0,0])
file_value = np.array([[0.2,0.4,0.4,0,0,0.5,0.5,0.2],[0.2,0.8,0.8,0,0,0.8,0.8,0.2],[0,0.3,0.5,0.6,0.6,0.5,0.3,0],[0,0.3,0.5,0.5,0.5,0.5,0.3,0]])
rank_value = np.array([[2, 1.7, 1.4, 1, 0.6, 0, 0,0],[2, 1.8, 1.5, 1.1, 0.6, 0, 0,0],[1.1, 1.3, 1.5, 1.7, 1.8, 1.6, 1.4, 1.2],[0.7, 1, 1.4, 1.7, 1.9, 2, 1.2,0.7]])
print(np.sum(file_value[:,2]*prediction + rank_value[:,2]*prediction))


2.3
